In [1]:
import json
import os
from os.path import join as pathjoin
import numpy as np
from PIL import Image

def saveForce(img,path):#不重复的存储图片
    respath = path
    for i in range(1000):
        if os.path.exists(respath) == True:
            dir,f=os.path.split(path)
            f='.'.join(f.split('.')[:-1])+str(i)+'.'+f.split('.')[-1]
            respath=pathjoin(dir,f)
            # print(path)
        else:
            break
    img.save(respath)
def splitExp(imagesPath,labelsPath,savePath):
    # labelsPath=r'D:\Desktop\1.WheatCounting\Detect\yolov5-master\yolov5-master\runs\MyOperation\SAM_labels\labels\train'
    # imagePath=r'D:\Desktop\1.WheatCounting\Detect\yolov5-master\yolov5-master\runs\MyOperation\SAM_labels\images\train'
    # savePath='./AfterSpliting'#todo 保存分割后图像的路径
    imgs=[Image.open(pathjoin(imagesPath,img)) for img in os.listdir(imagesPath) if not len(img.split('.')) ==1 ]#所有图片（原图）
    labels=[x for x in os.listdir(labelsPath) if  x.find('classes') == -1]#所有标签
    for i,txt in enumerate(labels):
        try:
            if not txt.find('class') == -1:
                continue
            posPath=pathjoin(labelsPath,txt)
            img=imgs[i]#当前标签对应的图片
            imgSize=img.size

            with open(posPath, "r") as f:
                for s in f.readlines():#每个框的坐标
                    points=list(map(float,s.split()))[1:]#(center.x,center.y,width,height),中心坐标和宽高
                    leftup=(np.array((points[0],points[1]))*np.array([imgSize[0],imgSize[1]])).astype(int)#左上坐标（x,y)
                    wh=((np.array((points[2],points[3]))*np.array([imgSize[0],imgSize[1]])).astype(int))
                    leftup=leftup-(wh/2).astype(int)#左上角
                    zone=[leftup[0],leftup[1],leftup[0]+wh[0],leftup[1]+wh[1]]#裁剪区域
                    # print(zone)
                    detectedBox=img.crop(zone)
                    # print(pathjoin(savePath,'box'))
                    saveDir=pathjoin(savePath,f"{os.path.splitext(txt)[0]}")
                    if os.path.exists(saveDir) ==False:
                        os.mkdir(saveDir)
                    saveForce(detectedBox,pathjoin(saveDir,f'{txt.split(".")[0]}_box.jpg'))#保存检测后的子图到对应labels名称目录里
        except Exception as e:
            print(f"保存{txt}时出错\n{e}")

In [80]:
imagesPath = r'./images/train'

imgs = [
    Image.open(os.path.join(imagesPath, img))
    for img in os.listdir(imagesPath)
    if os.path.splitext(img)[1] and 'classes' not in img
]
print(imgs)

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x585 at 0x1DC2889CF70>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1000x585 at 0x1DC2CACAE50>]


In [34]:
imagesPath = './test/img/'
labelsPath = './test/label/'
savePath = './test/result/'

imgs=[Image.open(pathjoin(imagesPath,img)) for img in os.listdir(imagesPath) if not len(img.split('.')) ==1 ]#所有图片（原图）
labels=[x for x in os.listdir(labelsPath) if  x.find('classes') == -1]#所有标签
classes = []
with open(labelsPath + 'classes.txt', "r") as f:
    for c in f.readlines():
        classes.append(c.replace('\n', ''))
print(classes)


for i,txt in enumerate(labels):
    try:
        if not txt.find('class') == -1:
            continue
        posPath=pathjoin(labelsPath,txt)
        img=imgs[i]#当前标签对应的图片
        imgSize=img.size
        Prefabs = {}


        with open(posPath, "r") as f:
            for s in f.readlines():#每个框的坐标
                print('s:',s,)
                group = classes[int(s.split()[0])]
                print(classes[int(s.split()[0])])
                points=list(map(float,s.split()))[1:]#(center.x,center.y,width,height),中心坐标和宽高
                print('points:',points)
                leftup=(np.array((points[0],points[1]))*np.array([imgSize[0],imgSize[1]])).astype(int)#左上坐标（x,y)    #中心座標G
                print('leftup:',leftup)
                wh=((np.array((points[2],points[3]))*np.array([imgSize[0],imgSize[1]])).astype(int))
                print('wh:',wh)
                leftup=leftup-(wh/2).astype(int)#左上角
                zone=[leftup[0],leftup[1],leftup[0]+wh[0],leftup[1]+wh[1]]#裁剪区域
                print('zone', zone)
                print('center', (leftup[0] + 0.5 * wh[0]), (leftup[1] + 0.5 * wh[1]))
                detectedBox=img.crop(zone)
                # print(pathjoin(savePath,'box'))
                saveDir=pathjoin(savePath,f"{os.path.splitext(txt)[0]}")
                if os.path.exists(saveDir) ==False:
                    os.mkdir(saveDir)
                saveForce(detectedBox,pathjoin(saveDir,f'{txt.split(".")[0]}_box.jpg'))#保存检测后的子图到对应labels名称目录里
                # saveForce(detectedBox, saveDir + )#保存检测后的子图到对应labels名称目录里
                print('-'*30)
    except Exception as e:
        print(f"保存{txt}时出错\n{e}")

['Prefab_01', 'Prefab_01_children', 'Prefab_02', 'Prefab_02_children', 'Prefab_03', 'Prefab_03_children', 'Prefab_04', 'Prefab_04_children', 'Prefab_05', 'Prefab_05_children']
s: 0 0.162891 0.534188 0.146094 0.747009

Prefab_01
points: [0.162891, 0.534188, 0.146094, 0.747009]
leftup: [208 312]
wh: [187 437]
zone [115, 94, 302, 531]
center 208.5 312.5
------------------------------
s: 1 0.170703 0.225641 0.127344 0.112821

Prefab_01_children
points: [0.170703, 0.225641, 0.127344, 0.112821]
leftup: [218 131]
wh: [163  66]
zone [137, 98, 300, 164]
center 218.5 131.0
------------------------------
s: 1 0.155859 0.349573 0.114844 0.107692

Prefab_01_children
points: [0.155859, 0.349573, 0.114844, 0.107692]
leftup: [199 204]
wh: [147  62]
zone [126, 173, 273, 235]
center 199.5 204.0
------------------------------
s: 1 0.154688 0.471795 0.115625 0.105983

Prefab_01_children
points: [0.154688, 0.471795, 0.115625, 0.105983]
leftup: [198 276]
wh: [148  62]
zone [124, 245, 272, 307]
center 198.0 

In [74]:
imagesPath = './test/img/'
labelsPath = './test/label/'
savePath = './test/result/'

imgs=[Image.open(pathjoin(imagesPath,img)) for img in os.listdir(imagesPath) if not len(img.split('.')) ==1 ]#所有图片（原图）
labels=[x for x in os.listdir(labelsPath) if  x.find('classes') == -1]#所有标签
classes = []
with open(labelsPath + 'classes.txt', "r") as f:
    for c in f.readlines():
        classes.append(c.replace('\n', ''))
print(classes)


for i, txt in enumerate(labels):
    try:
        if not txt.find('class') == -1:
            continue
        posPath=pathjoin(labelsPath,txt)
        img=imgs[i]#当前标签对应的图片
        imgSize=img.size
        Prefabs = {}

        groups = []
        with open(posPath, "r") as f:
            for index, s in enumerate(f.readlines()):#每个框的坐标
                print('s:',s,)
                group = classes[int(s.split()[0])]
                groups.append(group)
                if 'children' not in group:
                    create_group = True
                    Prefabs[group] = {}
                    print(group)
                    points=list(map(float,s.split()))[1:]#(center.x,center.y,width,height),中心坐标和宽高
                    print('points:',points)
                    leftup=(np.array((points[0],points[1]))*np.array([imgSize[0],imgSize[1]])).astype(int)#左上坐标（x,y)    #中心座標G
                    Prefabs[group]["name"] = group + '.prefab'
                    Prefabs[group]["type"] = "prefab"
                    Prefabs[group]["position"] = [int(leftup[0]), int(leftup[1])] #prefab_format
                    print('leftup:',leftup)
                    wh=((np.array((points[2],points[3]))*np.array([imgSize[0],imgSize[1]])).astype(int))
                    print('wh:',wh)
                    leftup=leftup-(wh/2).astype(int)#左上角
                    zone=[leftup[0],leftup[1],leftup[0]+wh[0],leftup[1]+wh[1]]#裁剪区域
                    print('zone', zone)
                    print('center', (leftup[0] + 0.5 * wh[0]), (leftup[1] + 0.5 * wh[1]))
                    detectedBox=img.crop(zone)
                    # print(pathjoin(savePath,'box'))
                    saveDir=pathjoin(savePath,f"{os.path.splitext(txt)[0]}")
                    if os.path.exists(saveDir) ==False:
                        os.mkdir(saveDir)
                    # saveForce(detectedBox,pathjoin(saveDir,f'{txt.split(".")[0]}_Image.jpg'))#保存检测后的子图到对应labels名称目录里
                    saveForce(detectedBox, saveDir + '/' + txt.split(".")[0] +'_'+ group +'_Image'+ str(index) +'.jpg')#保存检测后的子图到对应labels名称目录里

                elif 'children' in group:
                    children = {}
                    groupName = '_'.join(group.split('_')[0:2])
                    print(groupName, groups, '----')
                    if groupName in groups and create_group:
                        print(groupName, groups)
                        Prefabs[groupName]["children"] = []
                        create_group = False
                    points=list(map(float,s.split()))[1:]#(center.x,center.y,width,height),中心坐标和宽高
                    print('points:',points)
                    leftup=(np.array((points[0],points[1]))*np.array([imgSize[0],imgSize[1]])).astype(int)#左上坐标（x,y)    #中心座標G
                    print('leftup:',leftup)
                    wh=((np.array((points[2],points[3]))*np.array([imgSize[0],imgSize[1]])).astype(int))
                    print('wh:',wh)
                    leftup=leftup-(wh/2).astype(int)#左上角
                    zone=[leftup[0],leftup[1],leftup[0]+wh[0],leftup[1]+wh[1]]#裁剪区域
                    print('zone', zone)
                    print('center', (leftup[0] + 0.5 * wh[0]), (leftup[1] + 0.5 * wh[1]))
                    detectedBox=img.crop(zone)
                    # print(pathjoin(savePath,'box'))
                    saveDir=pathjoin(savePath,f"{os.path.splitext(txt)[0]}")
                    if os.path.exists(saveDir) == False:
                        os.mkdir(saveDir)
                    # saveForce(detectedBox,pathjoin(saveDir,f'{txt.split(".")[0]}_'+ group +'_Image', str(index) ,'.jpg'))#保存检测后的子图到对应labels名称目录里               
                    saveForce(detectedBox, saveDir + '/' + txt.split(".")[0] +'_'+ groupName +'_Image'+ str(index) +'.jpg')#保存检测后的子图到对应labels名称目录里
                    children["name"] = saveDir + '/' + txt.split(".")[0] +'_'+ groupName +'_Image'+ str(index)+'.jpg'
                    children["type"] = "sprite"
                    children["position"] = [int(leftup[0]), int(leftup[1])]
                    print(Prefabs[groupName]["children"])
                    Prefabs[groupName]["children"].append(children)

                # saveForce(detectedBox, saveDir + )#保存检测后的子图到对应labels名称目录里
                print('-'*30)
    except Exception as e:
        print(f"保存{txt}时出错\n{e}")

['Prefab_01', 'Prefab_01_children', 'Prefab_02', 'Prefab_02_children', 'Prefab_03', 'Prefab_03_children', 'Prefab_04', 'Prefab_04_children', 'Prefab_05', 'Prefab_05_children']
s: 0 0.162891 0.534188 0.146094 0.747009

Prefab_01
points: [0.162891, 0.534188, 0.146094, 0.747009]
leftup: [208 312]
wh: [187 437]
zone [115, 94, 302, 531]
center 208.5 312.5
------------------------------
s: 1 0.170703 0.225641 0.127344 0.112821

Prefab_01 ['Prefab_01', 'Prefab_01_children'] ----
Prefab_01 ['Prefab_01', 'Prefab_01_children']
points: [0.170703, 0.225641, 0.127344, 0.112821]
leftup: [218 131]
wh: [163  66]
zone [137, 98, 300, 164]
center 218.5 131.0
[]
------------------------------
s: 1 0.155859 0.349573 0.114844 0.107692

Prefab_01 ['Prefab_01', 'Prefab_01_children', 'Prefab_01_children'] ----
points: [0.155859, 0.349573, 0.114844, 0.107692]
leftup: [199 204]
wh: [147  62]
zone [126, 173, 273, 235]
center 199.5 204.0
[{'name': './test/result/IGS/IGS_Prefab_01_Image1.jpg', 'type': 'sprite', 'po

In [75]:
Prefabs

{'Prefab_01': {'name': 'Prefab_01.prefab',
  'type': 'prefab',
  'position': [208, 312],
  'children': [{'name': './test/result/IGS/IGS_Prefab_01_Image1.jpg',
    'type': 'sprite',
    'position': [137, 98]},
   {'name': './test/result/IGS/IGS_Prefab_01_Image2.jpg',
    'type': 'sprite',
    'position': [126, 173]},
   {'name': './test/result/IGS/IGS_Prefab_01_Image3.jpg',
    'type': 'sprite',
    'position': [124, 245]},
   {'name': './test/result/IGS/IGS_Prefab_01_Image4.jpg',
    'type': 'sprite',
    'position': [126, 317]},
   {'name': './test/result/IGS/IGS_Prefab_01_Image5.jpg',
    'type': 'sprite',
    'position': [124, 389]},
   {'name': './test/result/IGS/IGS_Prefab_01_Image6.jpg',
    'type': 'sprite',
    'position': [122, 462]}]},
 'Prefab_02': {'name': 'Prefab_02.prefab',
  'type': 'prefab',
  'position': [472, 333],
  'children': [{'name': './test/result/IGS/IGS_Prefab_02_Image8.jpg',
    'type': 'sprite',
    'position': [323, 117]},
   {'name': './test/result/IGS/IGS

In [76]:
import json

result = {}
result["name"] = txt+ ".node"
result["type"] = "node"
result["children"] = []
result["children"].append(Prefabs)

result

print(savePath + txt.split('.')[0] +'.json')
with open(savePath + txt.split('.')[0] +'.json', 'w', encoding= 'utf-8') as f:
    f.write(json.dumps(result, ensure_ascii=False))


./test/result/IGS.json


In [21]:
a = 'Prefab_01_01'
print('_'.join(a.split('_')[0:2]))

Prefab_01


In [14]:
with open(savePath+ 'test.json', "w", encoding= 'utf-8') as f:

    Messages = {
        "name": 'IMG.node',
        "type": "node",
        "position":[0,0],
        "children": [
            {"name": "Prefab_01.prefab", 
             "type": "prefab",
             "children": [
            {     
                 "name" : "Image_01.png",
                "type": "node",
                "position":[0,0]
            } ,
            {   "name" : "Image_01.png",
                "type": "node",
                "position":[0,0]}
            ]
            },
            {"name": "Prefab_01.prefab", 
             "type": "prefab",
             "children": [
            {     
                 "name" : "Image_01.png",
                "type": "node",
                "position":[0,0]
            } ,
            {   "name" : "Image_01.png",
                "type": "node",
                "position":[0,0]}
            ]
            }
        ]
    }
    f.write(json.dumps(Messages, ensure_ascii=False))